In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import tkinter as tk
from tkinter import filedialog
from tensorflow import keras
from tensorflow.keras import layers, callbacks
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
from PIL import Image, ImageTk

In [2]:
# Dataset path
dataset_path = r"C:\Users\hp\OneDrive\Desktop\project research paper\project implementation\facial data"

In [3]:
# Categories (labels)
categories = ["Autistic", "Non_Autistic"]
img_size = 128  # Resize images to 128x128

In [5]:
def load_dataset():
    data = []
    labels = []
    for category in categories:
        path = os.path.join(dataset_path, category)
        label = categories.index(category)  # 0 for autistic, 1 for non-autistic
        for img_name in os.listdir(path):
            img_path = os.path.join(path, img_name)
            try:
                img = cv2.imread(img_path, cv2.IMREAD_COLOR)
                img = cv2.resize(img, (img_size, img_size))
                data.append(img)
                labels.append(label)
            except Exception as e:
                print(f"Error loading image {img_path}: {e}")
    return np.array(data), np.array(labels)


In [6]:
# Load dataset
data, labels = load_dataset()
data = data / 255.0  # Normalize pixel values (0-1)
labels_numeric = np.array(labels)  # Convert labels to categorical format

In [7]:
# Compute class weights to handle imbalance
class_weights = compute_class_weight(class_weight='balanced', classes=np.array([0, 1]), y=labels_numeric)
class_weights = {i: class_weights[i] for i in range(len(categories))}


In [8]:
# Convert labels to one-hot encoding
labels = keras.utils.to_categorical(labels_numeric, num_classes=2)

In [9]:
# Split dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)


In [10]:
# Data Augmentation
data_gen = keras.preprocessing.image.ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)

In [11]:
# Build Improved CNN Model
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(2, activation='softmax')  # 2 output classes
])

C:\Users\hp\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
# Learning Rate Scheduler
def scheduler(epoch, lr):
    return lr * 0.95 if epoch > 5 else lr

lr_callback = callbacks.LearningRateScheduler(scheduler)


In [13]:
# Compile the model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


In [15]:
# Train the model
epochs = 30
history = model.fit(data_gen.flow(X_train, y_train, batch_size=32), 
                    validation_data=(X_test, y_test), 
                    epochs=epochs, 
                    callbacks=[lr_callback])


Epoch 1/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 33s 463ms/step - accuracy: 0.5971 - loss: 2.2191 - val_accuracy: 0.4244 - val_loss: 2.9078 - learning_rate: 0.0010
Epoch 2/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 29s 451ms/step - accuracy: 0.5994 - loss: 0.6703 - val_accuracy: 0.4244 - val_loss: 1.0801 - learning_rate: 0.0010
Epoch 3/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 29s 452ms/step - accuracy: 0.6351 - loss: 0.6613 - val_accuracy: 0.5187 - val_loss: 0.7022 - learning_rate: 0.0010
Epoch 4/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 29s 450ms/step - accuracy: 0.6533 - loss: 0.6416 - val_accuracy: 0.4971 - val_loss: 0.7097 - learning_rate: 0.0010
Epoch 5/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 29s 454ms/step - accuracy: 0.6449 - loss: 0.6270 - val_accuracy: 0.5737 - val_loss: 0.6758 - learning_rate: 0.0010
Epoch 6/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 29s 450ms/step - accuracy: 0.6707 - loss: 0.6116 - val_accuracy: 0.5855 - val_loss: 0.6581 - learning_rate: 0.0010
Epoch 7/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 29s 453ms/step - accuracy: 0.7041 - loss: 0.

In [17]:
 #Save the trained model as a pickle file
with open("autism_cnn_model2.pkl", "wb") as f:
    pickle.dump(model, f)
print("Model training completed and saved as 'autism_cnn_model2.pkl'")


Model training completed and saved as 'autism_cnn_model2.pkl'


In [18]:
# Load the model
with open("autism_cnn_model2.pkl", "rb") as f:
    model = pickle.load(f)


In [22]:
# Tkinter GUI for Image Upload
def predict_image():
    file_path = filedialog.askopenfilename()
    if not file_path:
        return
    
    img = cv2.imread(file_path, cv2.IMREAD_COLOR)
    img = cv2.resize(img, (img_size, img_size))
    img = np.expand_dims(img, axis=0) / 255.0  # Normalize and reshape
    prediction = model.predict(img)
    class_index = np.argmax(prediction)
    result_label.config(text=f"Prediction: {categories[class_index]}")
    
     # Display selected image
    img = Image.open(file_path)
    img = img.resize((200, 200))
    img = ImageTk.PhotoImage(img)
    panel.config(image=img)
    panel.image = img


In [24]:
# GUI Setup
root = tk.Tk()
root.title("Autism Detection Model")
root.geometry("400x400")

title_label = tk.Label(root, text="Autism Detection", font=("Arial", 16))
title_label.pack(pady=10)

upload_button = tk.Button(root, text="Upload Image", command=predict_image)
upload_button.pack(pady=10)

result_label = tk.Label(root, text="", font=("Arial", 12))
result_label.pack(pady=10)

panel = tk.Label(root)
panel.pack()

root.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
